In [22]:
import pandas as pd
import unicodedata
df = pd.read_csv('data/rawData.csv')
df


,names,url,ingredients
0,Mie Goreng,https://panlasangpinoy.com/mie-goreng/,230 grams Egg Noodles
1,Mie Goreng,https://panlasangpinoy.com/mie-goreng/,4 ounces chicken breast sliced into serving pi...
2,Mie Goreng,https://panlasangpinoy.com/mie-goreng/,3 ounces shrimp deveined and shells removed
3,Mie Goreng,https://panlasangpinoy.com/mie-goreng/,3 eggs beaten
4,Mie Goreng,https://panlasangpinoy.com/mie-goreng/,1 ¼ cup mung bean sprouts
...,...,...,...
324,Cheesy Pork Caldereta,https://panlasangpinoy.com/cheesy-pork-caldereta/,4 cups water
325,Cheesy Pork Caldereta,https://panlasangpinoy.com/cheesy-pork-caldereta/,1 piece onion chopped
326,Cheesy Pork Caldereta,https://panlasangpinoy.com/cheesy-pork-caldereta/,4 cloves garlic minced
327,Cheesy Pork Caldereta,https://panlasangpinoy.com/cheesy-pork-caldereta/,3 tablespoons cooking oil


In [23]:
import unicodedata
unicodedata.numeric('¼')
unicodedata.name('¼')

for a, row in df.iterrows():
    for char in row['ingredients']:
        if unicodedata.name(char).startswith('VULGAR FRACTION'):
            normalized = unicodedata.normalize('NFKC',char)
            df.iloc[a,2] = df.iloc[a,2].replace(char,normalized)

df['ingredients']

0                                  230 grams Egg Noodles
1      4 ounces chicken breast sliced into serving pi...
2            3 ounces shrimp deveined and shells removed
3                                          3 eggs beaten
4                            1 1⁄4 cup mung bean sprouts
                             ...                        
324                                         4 cups water
325                                1 piece onion chopped
326                               4 cloves garlic minced
327                            3 tablespoons cooking oil
328                         Salt and ground black pepper
Name: ingredients, Length: 329, dtype: object

In [24]:
# df["ingredients"]
# word = df.iloc[3]["ingredients"]
# word = word.split(" ")
# print(word)
# for x in df["ingredients"]:
#     x = x.split(" ")
#     print(x)
#     for y in x:
#         if y.isdigit():
#             print(f"this is a number: {y}")

In [35]:
import spacy
import re
base_model = spacy.load('en_core_web_sm')
measurements = re.compile(r'(grams|ounces|cup|inch|quart|tablespoon|teaspoon)s?') 
extracted = []

for x, row in df.iterrows():
    print('\r', "Extracting ingredient for row",x, end=" ")
    tokens = base_model(row['ingredients'])
    extract = ''
    for token in tokens:
        print(f"\nTOKEN:{token.text}\nTOKEN.DEP_:{token.dep_}\nTOKEN.POS:{token.pos_}\n---------------")
        # gets the ingrdient
#         if (token.dep_ in ['nsubj','ROOT']) and (token.pos_ in ['NOUN', 'PROPN']) and (not measurements.match(token.text)): 
#             for child in token.children:
#                 if (not measurements.match(child.text)) and (child.dep_ in ['amod', 'compound']): # removes the unit
#                     extract += child.text + ' '
#             extract += token.text + ' '
#     extracted.append(extract)
# print(extracted)

# clean_recipe = df[['names','url']]
# clean_recipe['ingredients'] = extracted

# nam = float('NaN')

# clean_recipe.replace("",nam,inplace=True)
# clean_recipe.dropna(subset=["ingredients"], inplace=True)

# clean_recipe

# # clean_recipe.to_csv("cleanData.csv")

 Extracting ingredient for row 0 
TOKEN:230
TOKEN.DEP_:nummod
TOKEN.POS:NUM
---------------

TOKEN:grams
TOKEN.DEP_:ROOT
TOKEN.POS:NOUN
---------------

TOKEN:Egg
TOKEN.DEP_:compound
TOKEN.POS:PROPN
---------------

TOKEN:Noodles
TOKEN.DEP_:dobj
TOKEN.POS:PROPN
---------------
 Extracting ingredient for row 1 
TOKEN:4
TOKEN.DEP_:nummod
TOKEN.POS:NUM
---------------

TOKEN:ounces
TOKEN.DEP_:compound
TOKEN.POS:NOUN
---------------

TOKEN:chicken
TOKEN.DEP_:compound
TOKEN.POS:NOUN
---------------

TOKEN:breast
TOKEN.DEP_:nsubj
TOKEN.POS:NOUN
---------------

TOKEN:sliced
TOKEN.DEP_:ROOT
TOKEN.POS:VERB
---------------

TOKEN:into
TOKEN.DEP_:prep
TOKEN.POS:ADP
---------------

TOKEN:serving
TOKEN.DEP_:amod
TOKEN.POS:VERB
---------------

TOKEN:pieces
TOKEN.DEP_:pobj
TOKEN.POS:NOUN
---------------
 Extracting ingredient for row 2 
TOKEN:3
TOKEN.DEP_:nummod
TOKEN.POS:NUM
---------------

TOKEN:ounces
TOKEN.DEP_:compound
TOKEN.POS:NOUN
---------------

TOKEN:shrimp
TOKEN.DEP_:nsubj
TOKEN.POS:N

In [16]:
ingri = df['ingredients']
# ingri = base_model(ingri)
# for word in ingri.ents:
#     print(word.text,word.label_)
words = "230 gram of Egg Noodles"
text = base_model(words)
for word in text:
    
    print(word.text, word.dep_)

230 nummod
gram ROOT
of prep
Egg compound
Noodles pobj
